# Autoencoder

(Si no tienes GPU puedes [ejecutar este notebook en COLAB](https://colab.research.google.com/github/AlbertoRuiz/umucv/blob/master/notebooks/bottleneck.ipynb))

## Load MNIST data

In [ ]:
import os
os.environ["KERAS_BACKEND"] = "torch"

In [ ]:
import matplotlib.pyplot as plt
import numpy             as np

import keras

In [ ]:
(kxl,cl), (kxt,ct) = keras.datasets.mnist.load_data()

xl = kxl.reshape(len(kxl),-1)/255
xt = kxt.reshape(len(kxt),-1)/255
print(xl.shape, cl.shape)
print(xt.shape, ct.shape)

## 2D embedding

In [ ]:
model = keras.Sequential(
    [keras.Input(shape=(28*28,))] +
    [keras.layers.Dense(u, activation='relu' if u!=2 else 'sigmoid') for u in [256,128,64,32,2,32,64,128,256]]+
    [keras.layers.Dense(28*28, activation='linear')]
)

model.summary()

In [ ]:
model.compile(loss="mse", optimizer="adam")

In [ ]:
history = model.fit(xl, xl, epochs=100, batch_size=500)

In [ ]:
plt.plot(history.history['loss']);

In [ ]:
pred = model.predict(xl)

In [ ]:
def shdig(v):
    x = np.reshape(v,[28,28])
    plt.imshow(1-x, 'gray', vmin=0, vmax=1, interpolation="nearest");

def compare(k):
    plt.figure(figsize=(8,4))
    plt.subplot(1,2,1); shdig(xl[k])
    plt.subplot(1,2,2); shdig(pred[k])

In [ ]:
compare(37)

In [ ]:
compare(10235)

In [ ]:
encoder = keras.Sequential()
encoder.add(keras.Input(shape=(28*28,)))
for x in model.layers[:5]:
    encoder.add(x)
    
decoder = keras.Sequential()
decoder.add(keras.Input(shape=(2,)))
for x in model.layers[5:]:
    decoder.add(x)

In [ ]:
xy = encoder.predict(xl)

In [ ]:
plt.figure(figsize=(5,5))
plt.scatter(*xy.T,s=1,c=cl,cmap='tab10');

In [ ]:
ns = 1
plt.figure(figsize=(12,5))
for d in range(10):
    plt.subplot(2,5,ns)
    ns += 1
    plt.scatter(*xy.T,s=1,c=cl==d,cmap='coolwarm');
    plt.title(d)
    plt.axis('off')

## 3D embedding

In [ ]:
encoder = keras.Sequential()
encoder.add(keras.Input(shape=(28*28,)))
for u in [256, 128, 64, 32]:
    encoder.add(keras.layers.Dense(units=u, activation='relu'))
encoder.add(keras.layers.Dense(units=3, activation='sigmoid'))

decoder = keras.Sequential()
decoder.add(keras.Input(shape=(3,)))
for u in [32, 64, 128, 256]:
    decoder.add(keras.layers.Dense(units=u, activation='relu'))
decoder.add(keras.layers.Dense(units=28*28, activation='linear'))

model = keras.Sequential([encoder, decoder])
model.compile(loss='mse', optimizer='adam')

In [ ]:
encoder.summary()

In [ ]:
decoder.summary()

In [ ]:
history = model.fit(xl, xl, epochs=100, batch_size=500)

In [ ]:
plt.plot(history.history['loss']);

In [ ]:
xyz = encoder.predict(xl)

In [ ]:
showclass = 3

fig = plt.figure(figsize=(6,6))
ax = fig.add_subplot(projection='3d')
ax.plot(*xyz[cl!=showclass].T,'.',markersize=1,alpha=0.5,color='gray');
ax.plot(*xyz[cl==showclass].T,'.',markersize=1,alpha=0.5,color='red');

#ax.scatter(*map.T,s=0.1,c=cl==0,cmap='coolwarm',alpha=0.5);


In [ ]:
fig = plt.figure(figsize=(6,6))
ax = fig.add_subplot(projection='3d')
for c in range(10):
    ax.plot(*xyz[cl==c].T,'.',markersize=1,alpha=0.5,label=c);
plt.legend(markerscale=10);

Si estás en tu máquina local puedes visualizar dinámicamente el gráfico 3D con el backend tk de matplotlib:

In [ ]:
%matplotlib tk

In [ ]:
fig = plt.figure(figsize=(8,8))
ax = fig.add_subplot(projection='3d')
for c in range(10):
    ax.plot(*xyz[cl==c].T,'.',markersize=1,alpha=0.5,label=c);
plt.legend(markerscale=10);

Volvemos al los gráficos inline:

In [ ]:
%matplotlib inline

In [ ]:
from ipywidgets import interactive, FloatSlider

def fun(a=0.5,b=0.5,c=0.5):
    shdig(decoder.predict(np.array([[a,b,c]]),verbose=False));

interactive(fun, a=(0.,1,0.01), b=(0.,1,0.01), c=(0.,1,0.01))

Vemos que muchas posiciones del embedding producen imágenes que no tienen forma de dígitos reconocibles. Si queremos aprender un modelo de un conjunto de objetos que permita generar muestras realistas es necesario hacer alguna mejoras. Algunas técnicas para esto son los variational autoencoders y stable diffusion.